#Main file
This main file is based on the scripts that I had for minimizing the machines from Netlogo. The original was made to capture output from Behaviour Space, processs the machines and then print it to use in Stata. This is too cumbersome, so decided to implement and do everything in Python so I can centralise all the analysis and work on the next algorithms such as Joint Machines, frequencies and unused behavioural states in order to analyse properly how the transitions are happening.

#1 Receiving the Netlogo output
The first step is to load the files. Also, from the name of the file, save as variables the Global variables used for that output.

In [1]:
import pandas as pd
import numpy as np
import copy
import minimization as minz #My script in same folder for minimization routines

#Choose here the Globals and Name used for the experiment to load.
#Make sure the files exist (i.e. from Netlogo simulations)
experiment_clue = "trial"
n_states = 4
n_signals = 0
n_rounds = 5
N = 40
n_parents = 20

globals_list = (experiment_clue, n_states, n_signals, n_rounds, N, n_parents) #Save them as a list

#Experiment name based on the chosen experiment_clue and globals
chosen_experiment = "%s_states_%s_signal_%s_rounds_%s_N_%s_parents_%s.txt" % globals_list

#Both files have to use the same "chosen experiment" (to make sure come from the same simulation in Netlogo)
summary_file_name = "summary_" + chosen_experiment #Summary output
strategies_file_name = "strategies_" + chosen_experiment #Strategies output

#Path to Netlogo outputs
netlogo_folder = "/Users/luisalejandrolee/Desktop/\
autos_coordination_computational.git/netlogo_part/outputs/" #Netlogo outputs in this folder

#Read files and save them as data
summary_data = pd.read_csv(netlogo_folder + summary_file_name)
strategies_data = pd.read_csv(netlogo_folder + strategies_file_name)

#2 Minimise the automata
Use the functions to have a simple code here for minimising the auto and storing other relevant variables (as available states, etc)

In [5]:
#Other required globals
n_obs = 2 if n_signals == 0 else 4 #Define here (or change) the possible observations of the machines

#Minimise the machines. Uses algorithgm from "minimization" module
for netlogo_auto in strategies_data.auto_long: #For each auto (in Netlogo format)
    
    big_auto = minz.to_format_netlogo_auto(netlogo_auto) #Use function to convert the raw Netlogo auto in a list format
    init_state = big_auto[0] #Save initial state of the machine
    normal_auto = minz.new_empty_auto(n_obs, n_states) #Use function to create a new empty auto as a numpy array

    # Next block it to fill the new 'normal_auto' with the information from big_auto.
    # The objective is that normal_auto=big_auto but as an array (instead of a list)
    my_index = xrange(1, len(big_auto), n_obs + 1) # Each number in the index is where a state starts
    for i, j in enumerate(my_index):
        normal_auto['actions'][i] = big_auto[j]
        normal_auto['transitions'][i] = big_auto[j + 1:j + n_obs + 1]
        
    canon_auto = minz.convert_to_canonical(normal_auto, n_states, init_state, n_obs) #Use function for canonical form
    
    access_states = len(canon_auto) #n_states now is only the accesible states of the machine (before minimization)
    
    #Use function to get minimum behavioural equivalent auto
    #Passes "0" as 3rd argument because that's init_state now (always for canonincal auto)
    min_auto = minz.minimized_automaton(canon_auto, access_states, 0, n_obs)
    
    #print ("raw auto = ", netlogo_auto)
    #print ("cleaned auto = ", big_auto)
    #print ("array auto = ", normal_auto)
    #print ("canon auto = ", canon_auto)
    #print ("min auto = ", min_auto)


In [19]:
        min_auto = minimized_automaton() # Now minimize the auto
        #print 'min_auto = ', min_auto
        
        print_auto = np.array_str(min_auto) # The min_auto is passed as a string (not an array) for easy printing
        
        print_auto = print_auto.replace('[','') # Make the string less cluttered
        print_auto = print_auto.replace(']','')
        print_auto = print_auto.replace("'",'')
        print_auto = print_auto.replace(',',' ')
        print_auto = print_auto.replace('\n', '')
        #print 'print_auto = ', print_auto
        
        data.loc[ix, label] = print_auto # Print the minimized auto as a string into the dataframe
        
sorted_df = data.sort(['run_number', 'generation']) #Sort the data



